## Setup

In [6]:
import os
import pandas as pd

In [7]:
DATA_DIR = "../proc_data"
DATA_FILE = os.path.join(DATA_DIR, "processed.csv")

## Data Read

In [8]:
df = pd.read_csv(DATA_FILE)

In [9]:
df.shape

(170161, 11)

In [10]:
df.head()

,index,id,url,title,text,score,created_utc,subreddit,num_comments,upvote_ratio,over_18
0,189,1fxmtei,https://www.reddit.com/r/ucla/comments/1fxmtei...,Can I email back consulting clubs to ask for a...,My resume/application got rejected from a cons...,1,2024-10-06 18:10:40,ucla,0,1.00,False
1,190,1fxms88,https://i.redd.it/hvmfotrcf6td1.png,Got 100 days of free coffee! 🥲,A local Dunkin was giving away these away to t...,1,2024-10-06 18:09:19,DunkinDonuts,0,1.00,False
2,191,1fxmrv9,https://www.reddit.com/r/AffiliateCommunity/co...,30 percent commission! Easy sell,I’d love to invite you to join the AVY Coffee ...,1,2024-10-06 18:08:53,AffiliateCommunity,0,1.00,False
3,192,1fxmrnq,https://www.reddit.com/r/relationship_advice/c...,I (M28) confessed to my long time friend (F28)...,We’ve been friends for over 10 years. She brok...,1,2024-10-06 18:08:37,relationship_advice,1,1.00,False
4,193,1fxmr5d,https://www.reddit.com/r/CasualIreland/comment...,Is there a way to avoid having dinner with my ...,I don't like having dinner with my mam!! For s...,1,2024-10-06 18:08:00,CasualIreland,0,0.67,False
